In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torchvision
import PIL
from matplotlib import pyplot as plt

import utils
import datasets
from models import SRCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
div2k = datasets.Div2K(32, 2)
# lr, hr = div2k[0]
# print(lr.shape)
# utils.showImages([lr, hr])


In [3]:
dataloader = torch.utils.data.DataLoader(div2k, batch_size=16, num_workers=4)

model = SRCNN()

In [7]:
for i, batch in enumerate(dataloader):
    lr, hr = batch
    if i > 4:
        break
    with torch.no_grad():
        print(lr.shape)
        print(hr.shape)
        print(model(lr).shape)

torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 64, 64])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 64, 64])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 64, 64])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 64, 64])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 32, 32])
torch.Size([16, 3, 64, 64])
torch.Size([16, 3, 32, 32])
